<a href="https://colab.research.google.com/github/amhaiskar0921/BTTAIAmazonProject/blob/main/FinalComplementNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Complement Naive Bayes for Imbalaced Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score, classification_report

In [4]:
df_merged = pd.read_csv('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/preprocessed_merged_data.csv')
df_merged.head()

,product_id,product_locale,product_title,product_description,product_brand,esci_label,query,bert_input
0,B00BO9BGYU,es,light irridiance oxigenada 20vol 1000ml l negr...,NaN,light irridiance,I,yunsey tinte amoniaco,[CLS] yunsey tinte amoniaco [SEP] light irridi...
1,3836582864,es,peter lindbergh fashion photography 40th anniv...,NaN,taschen,I,libro channel decoracion,[CLS] libro channel decoracion [SEP] peter lin...
2,B085TCJ87R,us,lenovo thinkcentre m75s small form factor sff ...,NaN,lenovo,I,ryzen 11,[CLS] ryzen 11 [SEP] lenovo thinkcentr m75 sma...
3,B002T3RV5S,us,delta faucet foundations single function tub s...,NaN,delta faucet,I,tub faucet without shower head,[CLS] tub faucet without shower head [SEP] del...
4,B082KFTF41,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,E,educational toys 3 year old,[CLS] educ toy 3 year old [SEP] kokodi lcd wri...


In [5]:
df_merged.drop(columns=["bert_input", "product_id"], inplace=True)

In [6]:
df_merged.fillna("", inplace=True)
df_merged.head()

,product_locale,product_title,product_description,product_brand,esci_label,query
0,es,light irridiance oxigenada 20vol 1000ml l negr...,,light irridiance,I,yunsey tinte amoniaco
1,es,peter lindbergh fashion photography 40th anniv...,,taschen,I,libro channel decoracion
2,us,lenovo thinkcentre m75s small form factor sff ...,,lenovo,I,ryzen 11
3,us,delta faucet foundations single function tub s...,,delta faucet,I,tub faucet without shower head
4,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,E,educational toys 3 year old


##Data Prep

###Step 1: Combining relevant columns to create input column

In [7]:
# Create a new column for BERT input
df_merged['nb_input'] = ""

# Apply functions based on the language specified in the product_locale column
for index, row in df_merged.iterrows():
    locale = row['product_locale']
    query_text = row['query']
    title_text = row['product_title']
    description_text = row['product_description']
    brand_text = row['product_brand']


    nb_input = f'{query_text} {title_text} {description_text} {brand_text}'
    df_merged.at[index, 'nb_input'] = nb_input

###Step 2: Numerical mapping of label column

In [8]:
label_map = {"E": 0, "S": 1, "C": 2, "I": 3}

# Assuming df_merged is your DataFrame containing the "esci_label" column
df_merged["esci_label"] = df_merged["esci_label"].map(label_map)

In [9]:
df_merged.head()

,product_locale,product_title,product_description,product_brand,esci_label,query,nb_input
0,es,light irridiance oxigenada 20vol 1000ml l negr...,,light irridiance,3,yunsey tinte amoniaco,yunsey tinte amoniaco light irridiance oxigena...
1,es,peter lindbergh fashion photography 40th anniv...,,taschen,3,libro channel decoracion,libro channel decoracion peter lindbergh fashi...
2,us,lenovo thinkcentre m75s small form factor sff ...,,lenovo,3,ryzen 11,ryzen 11 lenovo thinkcentre m75s small form fa...
3,us,delta faucet foundations single function tub s...,,delta faucet,3,tub faucet without shower head,tub faucet without shower head delta faucet fo...
4,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,0,educational toys 3 year old,educational toys 3 year old kokodi lcd writing...


##One-hot encoding with TF-IDF and Count Vectorizer

In [10]:
y = df_merged['esci_label']
X = df_merged['nb_input']

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
X_test_count = scaler.fit_transform(X_test_count)

##Creating the model

###Training on TF-IDF data

In [14]:
# Create a Complement Naive Bayes classifier
cnb_model = ComplementNB()

In [15]:
# Train the model
cnb_model.fit(X_train_tfidf, y_train)

ComplementNB()

In [16]:
# Make predictions on the test set
y_pred_tfidf = cnb_model.predict(X_test_tfidf)

In [17]:
# Evaluate the best model using micro-averaging F1-score
micro_f1 = f1_score(y_test, y_pred_tfidf, average='micro')
print(f'Micro-Averaging F1-score (TF-IDF): {micro_f1}')

Micro-Averaging F1-score (TF-IDF): 0.5979760809567617


In [19]:
report = classification_report(y_test, y_pred_tfidf)
print('Classification Report:\n', report)

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.90      0.75       695
           1       0.26      0.07      0.10       259
           2       0.11      0.07      0.08        30
           3       0.22      0.06      0.09       103

    accuracy                           0.60      1087
   macro avg       0.31      0.27      0.26      1087
weighted avg       0.50      0.60      0.51      1087



###Training on Count Vectorizer data

In [27]:
cnb_model = ComplementNB()

In [28]:
# Train the model
cnb_model.fit(X_train_count, y_train)

ComplementNB()

In [35]:
# Make predictions on the test set
y_pred_count = cnb_model.predict(X_test_count)

In [36]:
micro_f1 = f1_score(y_test, y_pred_count, average='micro')
print(f'Micro-Averaging F1-score (Count): {micro_f1}')

Micro-Averaging F1-score (Count): 0.46274149034038636


In [37]:
report = classification_report(y_test, y_pred_count)
print('Classification Report:\n', report)

Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63       695
           1       0.27      0.25      0.26       259
           2       0.07      0.13      0.09        30
           3       0.09      0.12      0.10       103

    accuracy                           0.46      1087
   macro avg       0.27      0.28      0.27      1087
weighted avg       0.49      0.46      0.48      1087

